In [2]:
import pandas as pd
import numpy as np

In [3]:
ls

1.0-ildar-initial-data-exploration.ipynb


In [4]:
train_df = pd.read_csv('../data/raw/train_df.csv')

In [5]:
pd.options.display.max_columns = 100

In [6]:
train_df.head()

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True
1,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,1,147,198,147,198,63,78,129,81,114,111,66,144,129,147,True
2,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,2,144,192,144,192,57,75,126,78,108,105,60,138,126,141,True
3,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,3,165,207,165,207,39,90,114,75,78,117,51,153,129,141,True
4,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,4,162,204,162,204,33,84,111,72,72,114,45,150,123,138,True


In [7]:
train_df_descriptors = np.load('../data/raw/train_df_descriptors.npy')

In [8]:
train_df_descriptors.shape

(154420, 512)

In [9]:
train_df.shape

(154420, 21)

In [10]:
train_df_track_order_df = pd.read_csv('../data/raw/train_df_track_order_df.csv')

In [11]:
train_df_track_order_df.head()

,track_id,person_id
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621


In [12]:
train_df.is_val.value_counts()

False    115140
True      39280
Name: is_val, dtype: int64

In [13]:
len(np.unique(train_df.person_id.values))

935

In [14]:
len(np.unique(train_df.track_id.values))

15442

In [15]:
len(train_df_track_order_df.track_id.values)

15442

In [16]:
len(np.unique(train_df_track_order_df.track_id.values))

15442

In [17]:
# averaged train descriptors from the example
train_df_agg_descriptors = np.load('../models/baseline/train_df_agg_descriptors.npy')

In [18]:
train_df_agg_descriptors.shape

(15442, 512)

### hot to validate locally???
Reults on all data are not good 
100%|###########################################################################################################################################| 935/935 [06:44<00:00,  2.02it/s]
score 1 (tpr@fpr=1e-6): 0.3428 score 2 (mean distance): 0.8069
we have to see the results in sample subbission to ensure that we have similar values of score 

it was added to train/dev data logic but not to scoring logic
lets seee how and add this

after fixes 

Train stage...
100%|###########################################################################################################################################| 702/702 [05:20<00:00,  1.94it/s]
score 1 (tpr@fpr=1e-6): 0.3427 score 2 (mean distance): 0.8048

Dev stage...
100%|###########################################################################################################################################| 233/233 [02:03<00:00,  2.53it/s]
score 1 (tpr@fpr=1e-6): 0.3249 score 2 (mean distance): 0.8132



In [31]:
is_val_df = train_df.groupby('person_id').agg({'is_val': lambda x: set(x).pop()}).reset_index(drop=False)
is_val_df.head(1)

,person_id,is_val
0,00050593d7333113d45bb44b39379621,True


In [33]:
train_df_track_order_df = pd.merge(train_df_track_order_df, is_val_df, on='person_id', how='left')
train_df_track_order_df.head(1)

,track_id,person_id,is_val
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621,True


In [49]:
train_df_track_order_df[train_df_track_order_df.is_val == False].head(1)

,track_id,person_id,is_val
20,1c19742c099d8e1d3beb1295561a908a,0042903df7513096ba0bda9f3a2b09bb,False


In [34]:
train_df_track_order_df.is_val.mean()

0.25437119544100506

In [38]:
ls ../data/raw

data_sample/                 train_df_track_order_df.csv
train_df.csv                 train_gt_descriptors.npy
train_df_descriptors.npy     train_gt_df.csv


In [39]:
train_gt_df = pd.read_csv('../data/raw/train_gt_df.csv')
train_gt_df.head(1)

,crop_path,warp_path,person_id,person_img_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4
0,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948,0,167,193,167,193,32,94,97,73,62,110,49,155,114,136


### From where do we get pairs???
Мы считаем по самому тупому методу - для каждого чувака берем ембединги по каждому его треку и сравниваем с каждой фоткой из его референс имеджесов и считаем что это класс 1.
Потом также сравниваем эмбеддинги по каждому треку с со всеми не его фотками и считаем что это класс 0
Далее строим roc-curve по этим дистанциям и меткам классов и смотрим значение в fpr = 10^(-6)

In [22]:
for p_id, subdf in train_df_track_order_df.groupby('person_id'):
    print(len(subdf))
    break

20


In [ ]:
train_gt_df = pd.read_csv('../data/raw/train_gt_df.csv')

In [20]:
train_gt_df.head()

NameError: name 'train_gt_df' is not defined

In [21]:
len(np.unique(train_df_track_order_df.person_id.values))

935

In [ ]:
# algorithm

agg_descr_arr = np.load(conf.predicted_descr_path)
test_track_order_df = pd.read_csv(conf.test_track_order_df)
test_gt_df = pd.read_csv(conf.test_gt_df_path)
gt_descr = np.load(conf.gt_descriptors_path)

dist_arr = []
labels_arr = []

#for each person in test_track_order_file
for curr_person_id, subdf in tqdm(test_track_order_df.groupby('person_id')):
    # get gt_df that person
    gt_subdf = test_gt_df[test_gt_df.person_id == curr_person_id]
    # for each track with THIS person 
    for idx in subdf.index.values:
        # for each gt image of that person
        for jdx in gt_subdf.index.values:
            # compute the norm of the diff vector on some aggregated descriptor of the track
            # by that trach id with the gt_descriptor of a single photo
            dist = np.linalg.norm(agg_descr_arr[idx] - gt_descr[jdx])
            # add this to distances
            dist_arr.append(dist)
            # add positive label
            labels_arr.append(1)
    
    # get gt_df of not this person (all others)
    gt_subdf = test_gt_df[test_gt_df.person_id != curr_person_id]
    # for each track with this person 
    for idx in subdf.index.values:
        # for each gt image of ANY other person
        for jdx in gt_subdf.index.values:
            # compute the norm of the diff vector on some aggregated descriptor (from the NNet)of the track
            # by that track id with the gt_descriptor of a single photo
            dist = np.linalg.norm(agg_descr_arr[idx] - gt_descr[jdx])
            # add this to distances
            dist_arr.append(dist)
            # add positive label
            labels_arr.append(0)

# further ispection on dataset containings

In [34]:
ls ../data/raw/

data_sample/                 train_df_track_order_df.csv
train_df.csv                 train_gt_descriptors.npy
train_df_descriptors.npy     train_gt_df.csv


In [55]:
train_df.head()


,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True
1,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,1,147,198,147,198,63,78,129,81,114,111,66,144,129,147,True
2,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,2,144,192,144,192,57,75,126,78,108,105,60,138,126,141,True
3,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,3,165,207,165,207,39,90,114,75,78,117,51,153,129,141,True
4,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,4,162,204,162,204,33,84,111,72,72,114,45,150,123,138,True


is_val is used for local validation, but i can use my own split. gt is here for all train faces

In [56]:
train_gt_df.head()

,crop_path,warp_path,person_id,person_img_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4
0,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948,0,167,193,167,193,32,94,97,73,62,110,49,155,114,136
1,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948,1,285,383,285,383,113,157,227,141,207,184,128,282,248,269
2,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948,2,183,235,183,235,48,110,128,107,90,137,56,186,129,182
3,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948,3,231,281,231,281,49,121,152,119,94,152,48,214,157,213
4,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948,4,220,285,220,285,53,131,152,119,106,169,57,214,166,203


In [57]:
train_gt_df.shape

(4675, 18)

In [58]:
np.unique(train_gt_df.person_id).shape

(935,)

In [59]:
train_df_track_order_df.head()

,track_id,person_id
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621


In [64]:
train_df_track_order_df.shape

(15442, 2)

In [60]:
for track_id in train_df_track_order_df.track_id.values:
    print(track_id)
    break

250f8b8ee05b39460dfd8f99457c4dc7


In [69]:
pd.merge(train_df_track_order_df, train_df[['person_id', 'is_val']], on='person_id', how='left').shape#[is_val == False]

(2944040, 4)

In [67]:
np.unique(train_df.person_id.values).shape

(935,)

In [68]:
train_df.head()

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True
1,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,1,147,198,147,198,63,78,129,81,114,111,66,144,129,147,True
2,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,2,144,192,144,192,57,75,126,78,108,105,60,138,126,141,True
3,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,3,165,207,165,207,39,90,114,75,78,117,51,153,129,141,True
4,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,4,162,204,162,204,33,84,111,72,72,114,45,150,123,138,True


In [ ]:
00050593d7333113d45bb44b39379621 250f8b8ee05b39460dfd8f99457c4dc7

In [73]:
np.unique(train_df[train_df.person_id == '00050593d7333113d45bb44b39379621'].track_id)

array(['02f4ef5e5edee78fc57c2110d7b5cc31',
       '17ad673a6722f857ddf7a7eafb017db8',
       '250f8b8ee05b39460dfd8f99457c4dc7',
       '393f2b85841bade6b30e476557ade2b8',
       '467035909e088bac170ac0a2418d46db',
       '4d1ed2e56e8fb3a4a07ab2608f1686d7',
       '5f5d2a3430f772548465f527e1cfb90e',
       '6c87f2313f20a01dae9655474eece12f',
       '71f337d6bdf4723a6e25b6d53c146011',
       '83d0df1195c88ba61bb14e1e4422ddd7',
       '93be3adedcbc2a1e20559d1116fdbb8c',
       '9bd8ccae26cbbd64bcc4acce00375f64',
       '9e75dcdf652c505d9f2ce193a8a97a09',
       '9efb14e0f108af6346b7f013c9b4bbad',
       'a01a124be27134c07d10d9bc8e97cd09',
       'da2e3d0b2af6025c834017717a604813',
       'e6179a9a641480ab37f6ddedd8780c27',
       'e76ae482524d4c4e19d69f558a2cf6b1',
       'f53c6de832c48eb35610f84d60409ed5',
       'fbee84b98a464d74f9ceb365ca1cdbea'], dtype=object)

In [82]:
for r, dff in train_df[train_df.person_id == '00050593d7333113d45bb44b39379621'].groupby('track_id'):
    print(dff.sample(1))
    break

                                             crop_path  \
118  00050593d7333113d45bb44b39379621/T0kZV5iFPO8/0...   

                                             warp_path  \
118  00050593d7333113d45bb44b39379621/T0kZV5iFPO8/0...   

                            person_id     video_id  \
118  00050593d7333113d45bb44b39379621  T0kZV5iFPO8   

                             track_id  frame_id  bbox_x  bbox_y  bbox_w  \
118  02f4ef5e5edee78fc57c2110d7b5cc31         8     206     226     206   

     bbox_h  x0  y0   x1  y1   x2   y2  x3   y3   x4   y4  is_val  
118     226  59  71  153  79  100  114  52  152  138  160    True  


In [81]:
train_gt_df

,crop_path,warp_path,person_id,person_img_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4
0,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948,0,167,193,167,193,32,94,97,73,62,110,49,155,114,136
1,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0001_...,cb0b5cb9792a436f8cc6f8d2efc26948,1,285,383,285,383,113,157,227,141,207,184,128,282,248,269
2,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0002_...,cb0b5cb9792a436f8cc6f8d2efc26948,2,183,235,183,235,48,110,128,107,90,137,56,186,129,182
3,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0003_...,cb0b5cb9792a436f8cc6f8d2efc26948,3,231,281,231,281,49,121,152,119,94,152,48,214,157,213
4,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0004_...,cb0b5cb9792a436f8cc6f8d2efc26948,4,220,285,220,285,53,131,152,119,106,169,57,214,166,203


In [85]:
from tqdm import tqdm
for person_id in tqdm(np.unique(train_df.person_id.values)):
    pass

100%|██████████| 935/935 [00:00<00:00, 525271.13it/s]


In [87]:
935 / 32

29.21875

In [89]:
train_df_track_order_df.head()


,track_id,person_id
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621


In [90]:
train_df_agg_descriptors.shape

(15442, 512)

In [91]:
train_df.shape

(154420, 21)

In [92]:
train_df.head()

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True
1,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,1,147,198,147,198,63,78,129,81,114,111,66,144,129,147,True
2,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,2,144,192,144,192,57,75,126,78,108,105,60,138,126,141,True
3,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,3,165,207,165,207,39,90,114,75,78,117,51,153,129,141,True
4,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,4,162,204,162,204,33,84,111,72,72,114,45,150,123,138,True


In [1]:
# train_df.warp_path.values.tolist()

#### inspectiong dataset for rnn training


In [51]:
train_df_descriptors.shape

(154420, 512)

In [53]:
train_df.shape

(154420, 21)

In [55]:
train_df.head(1)

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True


In [56]:
train_df_track_order_df.shape

(15442, 3)

In [57]:
train_df_track_order_df.head()

,track_id,person_id,is_val
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621,True
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621,True
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621,True
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621,True
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621,True


In [58]:
train_gt_df.shape

(4675, 18)

In [59]:
train_gt_descriptors = np.load('../data/raw/train_gt_descriptors.npy')
train_gt_descriptors.shape

(4675, 512)

In [60]:
train_df.head(1)

,crop_path,warp_path,person_id,video_id,track_id,frame_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,is_val
0,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621/DJGvVAbyBBc/2...,00050593d7333113d45bb44b39379621,DJGvVAbyBBc,250f8b8ee05b39460dfd8f99457c4dc7,0,141,195,141,195,60,75,126,81,108,108,63,141,123,144,True


In [61]:
train_gt_df.head(1)

,crop_path,warp_path,person_id,person_img_id,bbox_x,bbox_y,bbox_w,bbox_h,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4
0,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948/gt_imgs/0000_...,cb0b5cb9792a436f8cc6f8d2efc26948,0,167,193,167,193,32,94,97,73,62,110,49,155,114,136


In [62]:
train_df_track_order_df

,track_id,person_id,is_val
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621,True
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621,True
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621,True
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621,True
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621,True
5,9bd8ccae26cbbd64bcc4acce00375f64,00050593d7333113d45bb44b39379621,True
6,6c87f2313f20a01dae9655474eece12f,00050593d7333113d45bb44b39379621,True
7,9efb14e0f108af6346b7f013c9b4bbad,00050593d7333113d45bb44b39379621,True
8,e76ae482524d4c4e19d69f558a2cf6b1,00050593d7333113d45bb44b39379621,True
9,da2e3d0b2af6025c834017717a604813,00050593d7333113d45bb44b39379621,True


In [66]:
for id, (track_id, person_id) in train_df_track_order_df[['track_id', 'person_id']].iterrows():
    print(track_id, person_id)
    break

250f8b8ee05b39460dfd8f99457c4dc7 00050593d7333113d45bb44b39379621


In [65]:
train_df_track_order_df[['track_id', 'person_id']]

,track_id,person_id
0,250f8b8ee05b39460dfd8f99457c4dc7,00050593d7333113d45bb44b39379621
1,f53c6de832c48eb35610f84d60409ed5,00050593d7333113d45bb44b39379621
2,93be3adedcbc2a1e20559d1116fdbb8c,00050593d7333113d45bb44b39379621
3,83d0df1195c88ba61bb14e1e4422ddd7,00050593d7333113d45bb44b39379621
4,a01a124be27134c07d10d9bc8e97cd09,00050593d7333113d45bb44b39379621
5,9bd8ccae26cbbd64bcc4acce00375f64,00050593d7333113d45bb44b39379621
6,6c87f2313f20a01dae9655474eece12f,00050593d7333113d45bb44b39379621
7,9efb14e0f108af6346b7f013c9b4bbad,00050593d7333113d45bb44b39379621
8,e76ae482524d4c4e19d69f558a2cf6b1,00050593d7333113d45bb44b39379621
9,da2e3d0b2af6025c834017717a604813,00050593d7333113d45bb44b39379621


In [68]:
train_df_track_order_df.shape

(15442, 3)

In [73]:
train_df.groupby('track_id').agg({'warp_path': 'count'}).warp_path.value_counts()

10    15442
Name: warp_path, dtype: int64